# Урок 2. Парсинг HTML. BeautifulSoup
**1. Необходимо собрать информацию о вакансиях на вводимую должность с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:**

* **наименование вакансии**
* **предлагаемую зарплату (отдельно мин. и и отдельно макс.)**
* **ссылку на саму вакансию**
* **cайт откуда собрана вакансия**

**По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.**

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import time

In [2]:
REQUEST = 'data scientist'

HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
                         'AppleWebKit/537.36 (KHTML, like Gecko)' 
                         'Chrome/78.0.3904.108 Safari/537.36'}

Сначала получим данные с сайта **Head Hunters**

In [3]:
hh_link = 'https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text={}'.format(REQUEST.replace(' ', '+'))

Пишем функцию получения данных о вакансиях с конкретной страницы результатов поиска

In [4]:
def parseHHPage(data, page_num):  
#   time.sleep(1)
    link = hh_link + '&page=' + str(page_num) if page_num else hh_link

    html = requests.get(link, headers=HEADERS).text
    parsed_html = bs(html, 'lxml')  
    
    vacancy_list = parsed_html.findAll('div', {'data-qa': 'vacancy-serp__vacancy'})
    
    for vacancy in vacancy_list:
        vac_dict = {}
        title = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
        vac_dict['Title'] = title.text
        salary = vacancy.find('div', {'data-qa': 'vacancy-serp__vacancy-compensation'})

        vac_dict['Salary from'] = None
        vac_dict['Salary to'] = None
        vac_dict['Currency'] = None 
        if salary:
            salary = re.sub(r'\s+', '', salary.text)
            if not salary.find('от'):
                re_search = re.search(r'(\d+)(.+)', salary)
                if re_search:
                    vac_dict['Salary from'] = re_search.group(1)
                    vac_dict['Currency'] = re_search.group(2)
            elif not salary.find('до'):
                re_search = re.search(r'(\d+)(.+)', salary)
                if re_search:
                    vac_dict['Salary to'] = re_search.group(1)
                    vac_dict['Currency'] = re_search.group(2)
            else:
                re_search = re.search(r'(\d+)-(\d+)(.+)', salary)
                if re_search:
                    vac_dict['Salary from'] = re_search.group(1)
                    vac_dict['Salary to'] = re_search.group(2)
                    vac_dict['Currency'] = re_search.group(3)

        vac_dict['Link'] = title['href']
        vac_dict['Site'] = 'hh.ru'
        vac_dict['Location'] = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-address'}).text
        vac_dict['Company'] = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).text
        data.append(vac_dict)
        
    return parsed_html

Теперь вызываем функцию для нулевой страницы. Узнаем номер последней страницы.

In [5]:
data = []
first_page_html = parseHHPage(data, None)

page_numbers = first_page_html.findAll('a', {'data-page': True})
max_page = max([int(page['data-page']) for page in page_numbers]) if page_numbers else 0

for i in range(1, max_page + 1):
    parseHHPage(data, i)

In [6]:
df = pd.DataFrame(data)
# df[df['Salary from'].notnull() | df['Salary to'].notnull()]
df

,Title,Salary from,Salary to,Currency,Link,Site,Location,Company
0,Data Scientist,None,200000,руб.,https://hh.ru/vacancy/35010399?query=data%20sc...,hh.ru,"Санкт-Петербург, Площадь Восстания",ООО Цифроматика
1,Аналитик данных / Data Scientist,120000,180000,руб.,https://hh.ru/vacancy/32377243?query=data%20sc...,hh.ru,Москва,Открытые Технологии
2,Data Scientist,30000,None,грн.,https://hh.ru/vacancy/34867583?query=data%20sc...,hh.ru,"Киев, Берестейская",Червоний маркет
3,Data Scientist,None,None,None,https://hh.ru/vacancy/34663433?query=data%20sc...,hh.ru,"Минск, Могилевская","Wargaming, Minsk (Game Stream)"
4,Data Scientist,None,None,None,https://hh.ru/vacancy/34870483?query=data%20sc...,hh.ru,"Москва, Савеловская",Samsung Research Center
...,...,...,...,...,...,...,...,...
484,Архитектор BigData DWH,None,None,None,https://hh.ru/vacancy/34285564?query=data%20sc...,hh.ru,"Москва, Аэропорт","Mail.Ru Group, Решения для бизнеса"
485,DevOps Engineer for an AI Swiss Startup,2500,3500,USD,https://hh.ru/vacancy/34659932?query=data%20sc...,hh.ru,Санкт-Петербург,Assaia International AG
486,Facebook Ads Manager,None,None,None,https://hh.ru/vacancy/34909654?query=data%20sc...,hh.ru,"Минск, Площадь Ленина",DELVE
487,Technical Recruiter (Recruiting),None,None,None,https://hh.ru/vacancy/34741490?query=data%20sc...,hh.ru,Минск,ООО МэпДэйта
